This notebook calculates simple baselines for comparison as well as lower bound and upper bound on the performance.

Lower bound is calculated by measuring the metric on the naively printed elemented (so just greedy going in the direction of expected next state).

Upper bound is calculated by using oracle agent that has access to true corruption of the printer. We can't do better than this, because we have access to the true model

## Simulator

In [28]:
    
from scipy.io import loadmat
# Load the MATLAB file
corruptions = loadmat('scripts/Corruption_Simulation/add_dudw_data.mat');

array_corruptions = corruptions['all_data']
array_corruptions = array_corruptions[0:5000]
def get_next(x, corr_id, control_x):
    return x + control_x*array_corruptions[corr_id.astype(int)-1, x.round().astype(int)-1]

def get_next_from_row(data_row, controls=None):
    if controls is None:
        controls = (data_row[6], data_row[11])
    next_x = get_next(data_row[5], data_row[1], controls[0])
    next_y = get_next(data_row[10], data_row[2], controls[1])
    return next_x, next_y

def get_next_2d(state, corr_ids, controls):
    return (get_next(state[0], corr_ids[0], controls[0]), get_next(state[1], corr_ids[1], controls[1]))

In [29]:
data_row = data[0]
get_next_2d((data_row[5], data_row[10]), (data_row[1], data_row[2]), (data_row[6], data_row[11]))

(560.0, 482.52048987995823)

In [27]:
get_next_from_row(data[0])

(560.0, 482.52048987995823)

In [41]:
def find_the_best_next(state, corr_ids, next_optimal):
    from itertools import product
    
    
    cur_min=1999999999
    
    for control in product(range(-3, 4), range(-3, 4)):
        next_state = get_next_2d(state, corr_ids, control)
            
        dist = (next_optimal[0]-next_state[0])**2+(next_optimal[1]-next_state[1])**2
        if dist<cur_min:
            cur_min=dist
            opt_control = control
            chosen_next_state = next_state
        #print(dist, control)
            
    return opt_control, cur_min, chosen_next_state
    
        

In [35]:
find_the_best_next((data_row[5], data_row[10]), (data_row[1], data_row[2]), (560, 482.52))

41.332308888202704 (-3, -3)
25.150572277852714 (-3, -2)
13.592614617613524 (-3, -1)
6.658435907485145 (-3, 0)
4.348036147467574 (-3, 1)
6.661415337560811 (-3, 2)
13.598573477764855 (-3, 3)
38.91673338404437 (-2, -3)
22.734996773694373 (-2, -2)
11.177039113455185 (-2, -1)
4.242860403326805 (-2, 0)
1.9324606433092346 (-2, 1)
4.2458398334024725 (-2, 2)
11.182997973606517 (-2, 3)
37.467388081549146 (-1, -3)
21.28565147119915 (-1, -2)
9.72769381095996 (-1, -1)
2.7935151008315806 (-1, 0)
0.4831153408140097 (-1, 1)
2.796494530907247 (-1, 2)
9.733652671111292 (-1, 3)
36.984272980717506 (0, -3)
20.802536370367513 (0, -2)
9.244578710128325 (0, -1)
2.3103999999999445 (0, 0)
2.3998237349698535e-07 (0, 1)
2.313379430075611 (0, 2)
9.250537570279656 (0, 3)
37.467388081549146 (1, -3)
21.28565147119915 (1, -2)
9.72769381095996 (1, -1)
2.7935151008315806 (1, 0)
0.4831153408140097 (1, 1)
2.796494530907247 (1, 2)
9.733652671111292 (1, 3)
38.91673338404437 (2, -3)
22.734996773694373 (2, -2)
11.177039113455

((0, 1), 2.3998237349698535e-07, (560.0, 482.52048987995823))

In [30]:
from itertools import product
range(-3,4)

range(-3, 4)

In [ ]:
prod

In [6]:
from numpy import genfromtxt
data = genfromtxt('data/real_data/validation/ext_1/100638_1.csv', delimiter=',')

Columns

0. Label of the 3D Body (original name from the original data).
1. Corruption ID1 - Label of the fault on the x-axis (1, 2, 3, etc., each number corresponds to a different fault).
2. Corruption ID2 Label of the fault on the y-axis (1, 2, 3, etc., each number corresponds to a different fault).
3. Trajectory ID - Number of the layer. For example, if a body is composed by 100 layers, we only take 10 of them (1,11,21,...,91), and we label THAT laters (1,2,...,10).
4. Ideal x_i - if the trajectory would follow optimal trajectory, it would be here.
5. Achieved X_i Broken-trajectory x_i or in other words - where are we now.
6. Control on x (it can take the values -1, 0, or 1).
7. Ideal x_{i+1} (which is just [x_i + the control on x]).
8. Achieved X_{i+1}Broken-trajectory x_{i+1}.
9. Ideal y_i.
10. Achieved Y_i - Broken-trajectory y_i.
11. Control on y (it can take the values -1, 0, or 1).
12. Ideal y_{i+1} (which is just [y_i + the control on y]).
13. Achieved Y_{i+1} - Broken-trajectory y_{i+1}.


In [10]:
print('x_i',data[5,10])

x_i 488.616699495866


In [22]:
print('x_{i+1}, y_{i+1} according to the simulator: ', get_next_from_row(data[0]))

x_{i+1}, y_{i+1} according to the simulator:  (560.0, 482.52048987995823)


In [8]:
print('x_{i+1}, y_{i+1} according to the data: ', data[0, [8, 13]])

x_{i+1}, y_{i+1} according to the data:  [560.         482.52048988]


In [24]:
print('x_{i+1}, y_{i+1} according to the simulator: ', get_next_from_row(data[0],(1,0)))

x_{i+1}, y_{i+1} according to the simulator:  (560.6950648177196, 481.0)


-------

In [ ]:
for i in range(cur_trajectory.shape[0]):
    

In [10]:
data[:, 3]

array([1., 1., 1., ..., 6., 6., 6.])

In [11]:
import numpy as np
np.unique(data[:, 3], return_counts=True)

(array([1., 2., 3., 4., 5., 6.]), array([314, 317, 295, 320, 317, 314]))

In [12]:
TRAJ_ID = 3

In [16]:
cur_trajectory = data[data[:, 3] == TRAJ_ID]

In [42]:
cur_point = (cur_trajectory[0, 5], cur_trajectory[0, 10])
sum_distances = 0
for i in range(cur_trajectory.shape[0]):
    corr_id=(cur_trajectory[i, 1], cur_trajectory[i, 2])
    optimal_next=(cur_trajectory[i, 7], cur_trajectory[i, 12])
    _, dist, cur_point=find_the_best_next(cur_point, corr_id, optimal_next)
    sum_distances+=dist
    
print(sum_distances)

58447.07918739377


In [43]:
((cur_trajectory[:,7]-cur_trajectory[:,8])**2+(cur_trajectory[:,12]-cur_trajectory[:,13])**2).sum()

1006075.4705415417

In [44]:
cur_trajectory[-1]

array([ 1.00638000e+05,  1.90100000e+03,  4.83100000e+03,  3.00000000e+00,
        5.16000000e+02,  5.15616261e+02,  0.00000000e+00,  5.16000000e+02,
        5.15616261e+02,  6.10000000e+02,  6.10352619e+02, -1.00000000e+00,
        5.16000000e+02,  6.08889959e+02])

In [29]:
(cur_trajectory[:, 0] - cur_trajectory[:, 2])**2

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.09298547,
        0.09298547,  0.09298547,  0.39230173,  0.39230173,  0.39230173,
        0.39230173,  0.93099076,  0.93099076,  0.93099076,  0.93099076,
        1.74556554,  1.74556554,  1.74556554,  1.74556554,  2.87617623,
        2.87617623,  2.87617623,  4.36677262,  4.36677262,  4.36677262,
        4.36677262,  4.36677262,  6.26526161,  6.26526161,  6.26526161,
        6.26526161,  6.26526161,  6.26526161,  8.62365877,  8.62365877,
        8.62365877,  8.62365877,  8.62365877, 11.49823258, 11.49823258,
       11.49823258, 11.49823258, 11.49823258, 11.49823258, 14.94928176,
       14.94928176, 14.94928176, 14.94928176, 14.94928176, 19.02702602,
       19.02702602, 19.02702602, 19.02702602, 19.02702602, 19.02702602,
       23.33787404, 23.33787404, 23.33787404, 23.33787404, 23.33787404,
       27.81104821, 27.81104821, 27.81104821, 27.81104821, 27.81104821,
       27.81104821, 32.38321641, 32.38321641, 32.38321641, 32.38

In [27]:
cur_trajectory[0]

array([560.        , 481.        , 560.        , 481.        ,
         0.        ,   1.        , 560.        , 560.        ,
       560.        , 482.52048988])

In [28]:
cur_trajectory[1]

array([560.        , 482.        , 560.        , 482.52048988,
         0.        ,   1.        , 560.        , 560.        ,
       560.        , 484.04391305])

In [21]:
data[:, 8]

array([560.        , 560.        , 560.        , ..., 559.67703693,
       559.67703693, 559.67703693])

In [8]:
my_data[3]

array([1.00638000e+05, 1.90100000e+03, 4.83100000e+03, 1.00000000e+00,
       5.60000000e+02, 5.60000000e+02, 1.00000000e+00, 5.61000000e+02,
       5.60695065e+02, 4.84000000e+02, 4.85568761e+02, 1.00000000e+00,
       5.61000000e+02, 4.87093526e+02])

In [4]:
my_data[:, [4,5,6]]

array([[1.00638000e+05, 1.90100000e+03, 4.83100000e+03, 1.00000000e+00,
        5.60000000e+02, 5.60000000e+02, 0.00000000e+00, 5.60000000e+02,
        5.60000000e+02, 4.81000000e+02, 4.81000000e+02, 1.00000000e+00,
        5.60000000e+02, 4.82520490e+02],
       [1.00638000e+05, 1.90100000e+03, 4.83100000e+03, 1.00000000e+00,
        5.60000000e+02, 5.60000000e+02, 0.00000000e+00, 5.60000000e+02,
        5.60000000e+02, 4.82000000e+02, 4.82520490e+02, 1.00000000e+00,
        5.60000000e+02, 4.84043913e+02],
       [1.00638000e+05, 1.90100000e+03, 4.83100000e+03, 1.00000000e+00,
        5.60000000e+02, 5.60000000e+02, 0.00000000e+00, 5.60000000e+02,
        5.60000000e+02, 4.83000000e+02, 4.84043913e+02, 1.00000000e+00,
        5.60000000e+02, 4.85568761e+02],
       [1.00638000e+05, 1.90100000e+03, 4.83100000e+03, 1.00000000e+00,
        5.60000000e+02, 5.60000000e+02, 1.00000000e+00, 5.61000000e+02,
        5.60695065e+02, 4.84000000e+02, 4.85568761e+02, 1.00000000e+00,
        5.610

In [3]:
my_data.shape

(1877, 14)

In [2]:
my_data.shape

(42000000, 14)

In [3]:
my_data[:10]

array([[  1.        ,   1.        ,   1.        ,   1.        ,
        134.        , 134.        ,   1.        , 135.        ,
        134.74331118, 352.        , 352.        ,  -1.        ,
        135.        , 350.73059368],
       [  1.        ,   1.        ,   1.        ,   2.        ,
        135.        , 134.74331118,   0.        , 135.        ,
        134.74331118, 351.        , 350.73059368,  -1.        ,
        135.        , 349.4867406 ],
       [  1.        ,   1.        ,   1.        ,   3.        ,
        135.        , 134.74331118,   0.        , 135.        ,
        134.74331118, 350.        , 349.4867406 ,  -1.        ,
        135.        , 348.26969003],
       [  1.        ,   1.        ,   1.        ,   4.        ,
        135.        , 134.74331118,   0.        , 135.        ,
        134.74331118, 349.        , 348.26969003,  -1.        ,
        135.        , 347.08066523],
       [  1.        ,   1.        ,   1.        ,   5.        ,
        135.        

In [7]:
import numpy as np

In [11]:
np.unique(my_data[:,0]*8 + my_data[:, 1])

array([ 0.,  9., 10., 11., 12., 13., 14., 15.])

In [8]:
np.unique(my_data[:, 0])

array([0., 1.])

In [9]:
np.unique(my_data[:, 1])

array([0., 1., 2., 3., 4., 5., 6., 7.])